# Seq2Seq Example. - building Kor-En Translator

## Strategy
- Pretrain Decoder - Endcoder for each dataset (Korean, English)
  - Korean Encoder Training
  - English Decoder Training
  - English Encoder Training
  - Korean Decoder Training
- End-to-End Training - parallel corpus text dataset
  - Korean - English Translator
  - English - Korean Translator
- 


In [3]:
from model import KorEncoder
from data import KoEnParallel, Korean
from transformers import AutoTokenizer, GPT2TokenizerFast
from torch.utils import data

ko_tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
# en_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

ko_train_dataset = Korean(ko_tokenizer, 1024, 'train')
ko_val_dataset = Korean(ko_tokenizer, 1024, 'validation')

train_dataloader = data.DataLoader(ko_train_dataset, batch_size=10, collate_fn=ko_train_dataset)
val_dataloader = data.DataLoader(ko_val_dataset,batch_size=10, collate_fn=ko_val_dataset)
ko_encoder = KorEncoder(len(ko_tokenizer.vocab), 128, 128, 2, padding_id=ko_tokenizer.pad_token_id)


Using custom data configuration Moo--korean-parallel-corpora-152c18962d41c571
Found cached dataset csv (/home/fritzprix/.cache/huggingface/datasets/Moo___csv/Moo--korean-parallel-corpora-152c18962d41c571/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration Moo--korean-parallel-corpora-152c18962d41c571
Found cached dataset csv (/home/fritzprix/.cache/huggingface/datasets/Moo___csv/Moo--korean-parallel-corpora-152c18962d41c571/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
import pytorch_lightning as pl
from pytorch_lightning import loggers, callbacks

logger = loggers.TensorBoardLogger('logs')
model_checkout = callbacks.ModelCheckpoint('model/encoder', filename='model-{epoch}-{val_loss:.3f}')

trainer = pl.Trainer(logger=logger, accelerator='gpu', callbacks=[model_checkout])
trainer.fit(model=ko_encoder, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 6.6 M 
1 | enc       | GRU              | 494 K 
2 | fc        | LazyLinear       | 0     
3 | loss      | CrossEntropyLoss | 0     
-----------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
28.193    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


torch.Size([10, 49, 51200])
torch.Size([10, 54, 51200])


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

torch.Size([10, 49, 51200])
torch.Size([10, 54, 51200])
torch.Size([10, 41, 51200])
torch.Size([10, 32, 51200])
torch.Size([10, 35, 51200])
torch.Size([10, 32, 51200])
torch.Size([10, 55, 51200])
torch.Size([10, 51, 51200])
torch.Size([10, 71, 51200])
torch.Size([10, 76, 51200])
torch.Size([10, 54, 51200])
torch.Size([10, 42, 51200])
torch.Size([10, 46, 51200])
torch.Size([10, 59, 51200])
torch.Size([10, 44, 51200])
torch.Size([10, 57, 51200])
torch.Size([10, 53, 51200])
torch.Size([10, 55, 51200])
torch.Size([10, 40, 51200])
torch.Size([10, 36, 51200])
torch.Size([10, 44, 51200])
torch.Size([10, 30, 51200])
torch.Size([10, 60, 51200])
torch.Size([10, 67, 51200])
torch.Size([10, 42, 51200])
torch.Size([10, 47, 51200])
torch.Size([10, 58, 51200])
torch.Size([10, 35, 51200])
torch.Size([10, 36, 51200])
torch.Size([10, 30, 51200])
torch.Size([10, 60, 51200])
torch.Size([10, 40, 51200])
torch.Size([10, 55, 51200])
torch.Size([10, 42, 51200])
torch.Size([10, 44, 51200])
torch.Size([10, 44, 

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
